In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
from functools import reduce
from pathlib import Path

import numpy as np

from pyplume import utils, plotting, constants
from pyplume.dataloaders import load_geo_points
from pyplume.postprocess import ParticleResult

In [76]:
results = {
    "thredds_smoothn": "results/simulation_tijuana_hurrkay_alongshore_2022-10-31T13-18-57-smoothn/particlefile.nc",
    "Kay HFR": "results/simulation_tijuana_hurrkay_alongshore_2022-11-02T13-36-12-nosmoothn/particlefile.nc",
    "Kay OI": "results/simulation_tijuana_hurrkay_oi_alongshore_2022-10-31T13-18-58/particlefile.nc",
    "Henri HFR": "results/simulation_ny_hurrhenri_hfrnet_2022-11-16T16-18-48/particlefile.nc",
    "Henri HYCOM": "results/simulation_ny_hurrhenri_hycom_2022-11-16T16-18-48/particlefile.nc",
}
to_compare = ["Henri HFR", "Henri HYCOM"]
# to_compare = ["Kay HFR", "Kay OI"]
coast_lats, coast_lons = load_geo_points("data/coastOR2Mex_tijuana.mat")

# domain = constants.TIJUANA_RIVER_DOMAIN
domain = {
    "S": 40.2,
    "N": 40.7,
    "W": -74.1,
    "E": -73.2
}
# domain = {
#     "S": 38.5,
#     "N": 39.2,
#     "W": -75,
#     "E": -74
# }

res_path = Path(f"results/compare_density_{'_'.join(to_compare)}")
res_path.mkdir(exist_ok=True)

for key in results:
    results[key] = ParticleResult(results[key])

In [77]:
# common_timestamps = np.intersect1d(results[to_compare[0]].data_vars["time"], results[to_compare[1]].data_vars["time"])
if len(to_compare) <= 1:
    tstamps = results[to_compare[0]].data_vars["time"]
    tstamps = tstamps[~np.isnan(tstamps)]
    common_timestamps = np.sort(np.unique(tstamps))
else:
    common_timestamps = reduce(np.intersect1d, list(map(lambda x: results[x].data_vars["time"], to_compare)))

In [ ]:
for i, t in enumerate(common_timestamps):
    lats1, lons1 = results[to_compare[0]].get_positions_time(t, query="before")
    lats2, lons2 = results[to_compare[1]].get_positions_time(t, query="before")
    fig, (ax1, ax2) = plotting.carree_subplots((1,2), domain=domain)
    plotting.plot_particle_density(lats1, lons1, bins=100, ax=ax1, pmax=1, title=f"{to_compare[0]} cumulative density at\n{t.astype('datetime64[s]')}")
    # plotting.plot_coastline(coast_lats, coast_lons, ax=ax1, c="k")
    plotting.plot_particle_density(lats2, lons2, bins=100, ax=ax2, pmax=1, title=f"{to_compare[1]} cumulative density at\n{t.astype('datetime64[s]')}")
    # plotting.plot_coastline(coast_lats, coast_lons, ax=ax2, c="k")
    strnum = str(i).zfill(3)
    plotting.draw_plt(savefile=res_path / f"plot_{strnum}", fig=fig, fit=True, figsize=(12, 6))

In [81]:
# save plot as final timestamps

t = common_timestamps[-1]
for i, name in enumerate(to_compare):
    starts = np.unique(np.array([results[name].ds["lat"][:, 0], results[name].ds["lon"][:, 0]]).T, axis=0).T
    lats, lons = results[name].get_positions_time(t, query="before")
    fig, ax = plotting.carree_subplots((1,1), domain=domain, land=True)
    plotting.plot_particle_density(lats, lons, bins=50, ax=ax, pmax=0.76, title=f"{name} cumulative density at\n{t.astype('datetime64[s]')}")
    # plotting.plot_coastline(coast_lats, coast_lons, ax=ax, c="k")
    ax.scatter(starts[1], starts[0], marker="x", c="r")
    plotting.draw_plt(savefile=res_path / f"density_plot_final_{name}.png", fig=fig, fit=True, figsize=(7, 4.5))